UNIFICACION CON DATASET DE VARIABLE (AÑO 2025)

In [ ]:
import pandas as pd
import unicodedata
from pathlib import Path

# Rutas
PATH_GENERAL = Path(r"C:\Users\andre\OneDrive\Documentos\IMF\TFM\Nuevos CSV\descargas\inmuebles_unificado_total_final.csv")
PATH_VARIABLES = Path(r"C:\Users\andre\OneDrive\Documentos\IMF\TFM\Nuevos CSV\descargas\gentrificacion_madrid.csv")

ANIO_INMUEBLES = 2025


def std_text(s: str):
    if pd.isna(s): 
        return s
    s = str(s).strip().lower()
    s = ''.join(ch for ch in unicodedata.normalize('NFKD', s) if not unicodedata.combining(ch))
    s = s.replace("-", "").replace(" ", "")
    return s

# Cargar
df_gen = pd.read_csv(PATH_GENERAL)
df_var = pd.read_csv(PATH_VARIABLES)

# Asegurar columna 'distrito_std' 
df_gen["distrito_std"] = df_gen["distrito"].astype(str).map(std_text)
# Añadir 'anio' fijo (si ya existe una columna de año en tu general, usa esa)
df_gen["anio"] = ANIO_INMUEBLES

# Normalizar columna Distrito 'distrito_std'
df_var["distrito_std"] = df_var["Distrito"].astype(str).map(std_text)

# Asegurar tipo de año como int
if "Ano" in df_var.columns:
    df_var["Ano"] = pd.to_numeric(df_var["Ano"], errors="coerce").astype("Int64")
else:
    raise ValueError("El dataset de variables debe tener la columna 'Ano'.")

# Merge
# Clave: distrito_std + año
df_final = df_gen.merge(
    df_var,
    left_on=["distrito_std", "anio"],
    right_on=["distrito_std", "Ano"],
    how="left",
    suffixes=("", "_soc")
)

print("Unificación completada")
print("Filas:", len(df_final), "| Columnas:", len(df_final.columns))
print("Columnas clave presentes:",
      {"precio_num" in df_final.columns, "Precio_venta" in df_final.columns, "Precio_alquiler" in df_final.columns})
print(df_final[["distrito","anio","precio_num","Precio_venta","Precio_alquiler"]].head(5))

# Guardar
OUT_DIR = PATH_GENERAL.parent
df_final.to_csv(OUT_DIR / "inmuebles_+_variables_distrito_anio.csv", index=False)
df_final.to_pickle(OUT_DIR / "inmuebles_+_variables_distrito_anio.pkl")
try:
    df_final.to_parquet(OUT_DIR / "inmuebles_+_variables_distrito_anio.parquet", index=False)
except Exception as e:
    print("Parquet no guardado (instala pyarrow o fastparquet). Detalle:", e)

print("Guardado en:", OUT_DIR)


Unificación completada
Filas: 15960 | Columnas: 44
Columnas clave presentes: {True}
     distrito  anio  precio_num  Precio_venta  Precio_alquiler
0  arganzuela  2025      1280.0           NaN              NaN
1  arganzuela  2025      1700.0           NaN              NaN
2  arganzuela  2025      1300.0           NaN              NaN
3  arganzuela  2025      1500.0           NaN              NaN
4  arganzuela  2025      1000.0           NaN              NaN
Guardado en: C:\Users\andre\OneDrive\Documentos\IMF\TFM\Nuevos CSV\descargas


In [10]:
# Guardar en CSV
OUT_PATH = Path(r"C:\Users\andre\OneDrive\Documentos\IMF\TFM\Nuevos CSV\descargas\inmuebles_variables_final.csv")
df_final.to_csv(OUT_PATH, index=False, encoding="utf-8")

print("Dataset final guardado en CSV:", OUT_PATH)
print("Filas:", len(df_final), "| Columnas:", len(df_final.columns))

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\andre\\OneDrive\\Documentos\\IMF\\TFM\\Nuevos CSV\\descargas\\inmuebles_variables_final.csv'

ANALISIS DE DATOS

In [ ]:
df = df_final
# Estructura general
print("Dimensiones del dataset:", df.shape)
print("\nTipos de datos:")
print(df.dtypes)

Dimensiones del dataset: (16254, 44)

Tipos de datos:
descripcion                  object
descripcion_larga            object
distrito                     object
distrito_std                 object
habitaciones                 object
link                         object
localidad                    object
operacion                    object
origen_archivo               object
precio                       object
tamanio                      object
has_antigüedad               object
has_banos                    object
has_garaje                   object
has_trastero                 object
has_piscina                  object
has_terraza                  object
precio_num                  float64
num_banos                   float64
anio_construccion           float64
antiguedad_texto            float64
antiguedad_calc             float64
antiguedad_final            float64
texto_embedding              object
embedding_str                object
anio                          Int64
Ano       

In [34]:
# Comparación entre 'anio' y 'Ano'

print(" Valores únicos en 'anio':", sorted(df["anio"].dropna().unique()))
print(" Valores únicos en 'Ano':", sorted(df["Ano"].dropna().unique()))

# Ver si hay diferencias fila a fila
df_diff = df[df["anio"] != df["Ano"]][["distrito","anio","Ano","precio_num"]]

print("\n Filas donde 'anio' ≠ 'Ano':", len(df_diff))
print(df_diff.head(10))


 Valores únicos en 'anio': [2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025]
 Valores únicos en 'Ano': []

 Filas donde 'anio' ≠ 'Ano': 0
Empty DataFrame
Columns: [distrito, anio, Ano, precio_num]
Index: []


In [35]:
# Valores nulos

print("\n Porcentaje de valores nulos por columna:")
print((df.isna().mean() * 100).round(2).sort_values(ascending=False))


 Porcentaje de valores nulos por columna:
Ano                         100.00
Distrito                    100.00
Renta_bruta_hogar            98.97
Densidad_poblacion           98.97
Paro_registrado%             98.97
Tamano_vivienda_personas     98.97
Menores_18anos%              98.97
Mayores_65anos%              98.97
Edad_media                   98.97
Renta_bruta_persona          98.97
Renta_neta_hogar             98.97
Renta_neta_persona           98.97
Apartamentos_turisticos      98.71
Precio_alquiler              98.19
Precio_venta                 98.19
Zonas_verdes%                98.19
Esperanza_vida               98.19
Superficie_distrito_ha       98.19
antiguedad_calc              92.17
anio_construccion            92.17
antiguedad_texto             91.84
antiguedad_final             84.74
num_banos                    80.39
descripcion                  10.21
tamanio                       2.69
descripcion_larga             2.27
precio                        1.81
embedding_st

In [36]:
# Estadísticas numéricas

print("\n Estadísticas descriptivas numéricas:")
print(df.describe().T)


 Estadísticas descriptivas numéricas:
                            count           mean             std       min  \
precio_num                15960.0  550259.186779  1015089.042025    1000.0   
num_banos                  3187.0       1.657358        0.791479       1.0   
anio_construccion          1272.0    1994.226415       38.620089    1900.0   
antiguedad_texto           1327.0       0.026375        0.641734       0.0   
antiguedad_calc            1272.0      30.773585       38.620089       0.0   
antiguedad_final           2481.0      14.129786       30.037317       0.0   
anio                      16254.0    2024.864341        1.137122    2011.0   
Precio_venta                294.0    3118.847959     1213.657363   1246.75   
Precio_alquiler             294.0      12.621902        3.132282  7.533333   
Esperanza_vida              294.0      84.706122        1.276954      80.5   
Renta_neta_persona          168.0   17161.136905     5327.767736    9276.0   
Renta_neta_hogar         

In [37]:
# Variables categóricas clave

print("\n Distribución de 'operacion':")
print(df["operacion"].value_counts(dropna=False))

print("\n Ejemplo de distritos:")
print(df["distrito"].value_counts(dropna=False).head(21))


 Distribución de 'operacion':
operacion
venta       9613
alquiler    6347
NaN          294
Name: count, dtype: int64

 Ejemplo de distritos:
distrito
chamartin              1157
tetuan                 1146
chamberi               1123
barrio-de-salamanca    1112
centro                 1098
retiro                 1064
moncloa                1027
arganzuela              970
ciudad-lineal           912
fuencarral              836
hortaleza               803
carabanchel             763
puente-de-vallecas      723
latina                  658
san-blas                589
usera                   486
villaverde              446
vicalvaro               383
villa-de-vallecas       350
moratalaz               160
barajas                 154
Name: count, dtype: int64


PRUEBA INTERNA

In [39]:
import pandas as pd

# Ruta del archivo
path = r"C:\Users\andre\OneDrive\Documentos\IMF\TFM\Nuevos CSV\inmuebles_variables_panel_modificado.csv"

# Cargar dataset
df = pd.read_csv(path)

# Mostrar columnas
print("El dataset tiene", len(df.columns), "columnas.")
print("Lista de columnas:")
for col in df.columns:
    print("-", col)


El dataset tiene 44 columnas.
Lista de columnas:
- descripcion
- descripcion_larga
- distrito
- distrito_std
- habitaciones
- link
- localidad
- operacion
- origen_archivo
- precio
- tamanio
- has_antigüedad
- has_banos
- has_garaje
- has_trastero
- has_piscina
- has_terraza
- precio_num
- num_banos
- anio_construccion
- antiguedad_texto
- antiguedad_calc
- antiguedad_final
- texto_embedding
- embedding_str
- anio
- Ano
- Distrito
- Precio_venta
- Precio_alquiler
- Esperanza_vida
- Renta_neta_persona
- Renta_neta_hogar
- Renta_bruta_persona
- Renta_bruta_hogar
- Edad_media
- Mayores_65anos%
- Menores_18anos%
- Tamano_vivienda_personas
- Paro_registrado%
- Apartamentos_turisticos
- Superficie_distrito_ha
- Densidad_poblacion
- Zonas_verdes%


In [ ]:
import pandas as pd

# Ruta del archivo original
path_in = r"C:\Users\andre\OneDrive\Documentos\IMF\TFM\Nuevos CSV\inmuebles_variables_panel_modificado.csv"
path_out = r"C:\Users\andre\OneDrive\Documentos\IMF\TFM\Nuevos CSV\iinmuebles_variables_panel_modificado_clean.csv"

# Leer dataset
df = pd.read_csv(path_in)

# Columnas a eliminar
cols_to_drop = ["origen_archivo", "descripcion", "descripcion_larga", "link", "texto_embedding", "embedding_str"]

# Eliminar si existen
df = df.drop(columns=[c for c in cols_to_drop if c in df.columns])

# Guardar dataset limpio
df.to_csv(path_out, index=False, encoding="utf-8")

print("Dataset limpio guardado en:", path_out)
print("Columnas finales:", df.columns.tolist())


✅ Dataset limpio guardado en: C:\Users\andre\OneDrive\Documentos\IMF\TFM\Nuevos CSV\iinmuebles_variables_panel_modificado_clean.csv
Columnas finales: ['distrito', 'distrito_std', 'habitaciones', 'localidad', 'operacion', 'precio', 'tamanio', 'has_antigüedad', 'has_banos', 'has_garaje', 'has_trastero', 'has_piscina', 'has_terraza', 'precio_num', 'num_banos', 'anio_construccion', 'antiguedad_texto', 'antiguedad_calc', 'antiguedad_final', 'anio', 'Ano', 'Distrito', 'Precio_venta', 'Precio_alquiler', 'Esperanza_vida', 'Renta_neta_persona', 'Renta_neta_hogar', 'Renta_bruta_persona', 'Renta_bruta_hogar', 'Edad_media', 'Mayores_65anos%', 'Menores_18anos%', 'Tamano_vivienda_personas', 'Paro_registrado%', 'Apartamentos_turisticos', 'Superficie_distrito_ha', 'Densidad_poblacion', 'Zonas_verdes%']
